In [50]:
# ==============================
# 0. Install/Import
# ==============================
#!pip install google-generativeai python-dotenv tqdm pandas --quiet

from dotenv import load_dotenv
import os
from pathlib import Path
from IPython.display import display
import pandas as pd
import json
import re
import time
from tqdm.notebook import tqdm
import google.generativeai as genai

# ==============================
# 1. ENVIRONMENT SETUP
# ==============================
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY not set in your .env file!")
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.5-flash-preview-05-20")

In [51]:
SYSTEM_PROMPT = """
You are a medical data parser. Given a block of Taiwanese hospital lab report text, extract each individual test into a structured JSON object for API use.
Remove all names of patients, doctors, and lab staff for privacy.
Convert all dates from ROC (民國) to Gregorian (西元) format and merge with times as YYYYMMDD-HHMMSS.
For each lab test, extract:
patient_id (病歷號)
lab_order_id (單據號碼)
encounter_id (門診序號)
department (科別)
specimen (檢體別)
lab_report_type (⊙...⊙ section)
diagnosis_codes (all ICD codes listed)
dates: { requested, collected, received, reported }
tests: { test_name, value, unit, reference_range, interpretation }
If a test value has "H" or "L", set interpretation as "elevated" or "decreased", otherwise "normal" or use existing qualitative result (e.g., Reactive).
Return a JSON array. Do NOT include or return any names or national IDs.
Output only the JSON.
[
  {
    "name": "parse_lab_report",
    "description": "Parse a Taiwanese hospital lab report into structured JSON objects.",
    "parameters": {
      "type": "object",
      "properties": {
        "reports": {
          "type": "array",
          "description": "List of parsed lab report entries.",
          "items": {
            "type": "object",
            "properties": {
              "patient_id": {
                "type": "string"
              },
              "lab_order_id": {
                "type": "string"
              },
              "encounter_id": {
                "type": "string"
              },
              "department": {
                "type": "string"
              },
              "specimen": {
                "type": "string"
              },
              "lab_report_type": {
                "type": "string"
              },
              "diagnosis_codes": {
                "type": "array",
                "items": {
                  "type": "string"
                }
              },
              "dates": {
                "type": "object",
                "properties": {
                  "requested": {
                    "type": "string"
                  },
                  "collected": {
                    "type": "string"
                  },
                  "received": {
                    "type": "string"
                  },
                  "reported": {
                    "type": "string"
                  }
                }
              },
              "tests": {
                "type": "array",
                "items": {
                  "type": "object",
                  "properties": {
                    "test_name": {
                      "type": "string"
                    },
                    "value": {
                      "type": "string"
                    },
                    "unit": {
                      "type": "string"
                    },
                    "reference_range": {
                      "type": "string"
                    },
                    "interpretation": {
                      "type": "string"
                    }
                  }
                }
              }
            }
          }
        }
      }
    }
  }
]
"""


In [52]:
# 3. UTILS
# ==============================

def split_lab_blocks(text):
    # Split at ~I; or ~IL2X3; (delimiters)
    blocks = re.split(r'(?<=\n)(~I;|~IL2X3;)', text)
    out_blocks = []
    for i in range(1, len(blocks), 2):
        block = blocks[i] + blocks[i+1]
        if len(block.strip()) > 100:
            out_blocks.append(block.strip())
    return out_blocks

def extract_reports(text):
    """
    Robustly extracts the 'reports' list from various possible Gemini output structures.
    Supports:
    - [ {...}, {...} ]
    - { "reports": [...] }
    - { "parameters": { "reports": [...] } }
    - { ... } (single object that is a report)
    Returns a list of report dicts or [].
    """
    # Remove markdown/code block formatting if present
    text = text.strip()
    if text.startswith("```"):
        text = re.sub(r"^```[a-z]*\n?", "", text)
        text = re.sub(r"\n?```$", "", text)
        text = text.strip()
    try:
        data = json.loads(text)
    except Exception:
        # Try to recover from malformed JSON (not recommended but fallback)
        try:
            # Remove trailing commas (common LLM error)
            text = re.sub(r",(\s*[}\]])", r"\1", text)
            data = json.loads(text)
        except Exception:
            raise ValueError("No valid JSON object found.")
    # Handle various wrapping
    if isinstance(data, list):
        return data
    if isinstance(data, dict):
        if "parameters" in data and "reports" in data["parameters"]:
            return data["parameters"]["reports"]
        if "reports" in data:
            return data["reports"]
        # If single dict looks like a report (has keys)
        if "patient_id" in data:
            return [data]
    return []


def flatten_reports(json_data):
    flat_rows = []
    reports = json_data.get("reports", [])
    for report in reports:
        parent = {
            "patient_id": report.get("patient_id", ""),
            "lab_order_id": report.get("lab_order_id", ""),
            "encounter_id": report.get("encounter_id", ""),
            "department": report.get("department", ""),
            "specimen": report.get("specimen", ""),
            "lab_report_type": report.get("lab_report_type", ""),
            "diagnosis_codes": ";".join(report.get("diagnosis_codes", [])),
            "requested": report.get("dates", {}).get("requested", ""),
            "collected": report.get("dates", {}).get("collected", ""),
            "received": report.get("dates", {}).get("received", ""),
            "reported": report.get("dates", {}).get("reported", ""),
        }
        tests = report.get("tests", [])
        for test in tests:
            row = parent.copy()
            row.update({
                "test_name": test.get("test_name", ""),
                "value": test.get("value", ""),
                "unit": test.get("unit", ""),
                "reference_range": test.get("reference_range", ""),
                "interpretation": test.get("interpretation", ""),
            })
            flat_rows.append(row)
    return flat_rows

def call_gemini_api(block, model, sys_inst):
    prompt = sys_inst.strip() + "\n\n" + block.strip()
    try:
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(temperature=0),
            stream=False,
        )
        return response.text
    except Exception as e:
        print(f"[ERROR] Gemini API error: {e}")
        return None

def process_lab_txt_file(input_path, output_json_path, output_csv_path, model, batch_size=3, sleep_s=1):
    with open(input_path, encoding="utf-8") as f:
        raw = f.read()
    blocks = split_lab_blocks(raw)
    print(f"[INFO] Total lab sheets found: {len(blocks)}")

    merged_reports = []
    raw_gemini_outputs = []
    total_batches = (len(blocks) + batch_size - 1) // batch_size
    with tqdm(total=total_batches, desc="Gemini batches") as bar:
        for i in range(0, len(blocks), batch_size):
            batch = blocks[i:i+batch_size]
            for idx, block in enumerate(batch):
                output = call_gemini_api(block, model, SYSTEM_PROMPT)
                # --- Save raw Gemini reply regardless of parse ---
                if output is not None:
                    raw_gemini_outputs.append(
                        f"\n=== Gemini Reply for Block {i+idx+1} ===\n{output.strip()}\n"
                    )
                    try:
                        reports = extract_reports(output)
                        merged_reports.extend(reports)
                    except Exception as e:
                        print(f"[ERROR] Failed to parse Gemini output: {e}")
                else:
                    raw_gemini_outputs.append(
                        f"\n=== Gemini Reply for Block {i+idx+1} ===\n[API returned None]\n"
                    )
                time.sleep(sleep_s)  # Avoid rate limit
            bar.update(1)

    # Save raw Gemini replies to TXT for debug
    raw_txt_path = output_json_path.parent / (output_json_path.stem.replace("_parsed", "_gemini_raw_replies") + ".txt")
    with open(raw_txt_path, "w", encoding="utf-8") as f:
        f.writelines(raw_gemini_outputs)
    print(f"[INFO] Saved raw Gemini replies: {raw_txt_path}")

    # Save merged JSON
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump({"reports": merged_reports}, f, ensure_ascii=False, indent=2)
    print(f"[INFO] Saved merged JSON: {output_json_path}")

    # Flatten and save CSV
    flat_rows = flatten_reports({"reports": merged_reports})
    df = pd.DataFrame(flat_rows)
    df.to_csv(output_csv_path, index=False)
    print(f"[INFO] Saved flattened CSV: {output_csv_path}")
    return df




In [53]:
# 4. EXECUTION BLOCK
# ==============================

input_dir = Path("./input")
output_dir = Path("./output")
output_dir.mkdir(exist_ok=True)

input_files = list(input_dir.glob("*.txt"))
print(f"[INFO] Found {len(input_files)} txt files: {[f.name for f in input_files]}")
if not input_files:
    print("[INFO] No .txt files found in input directory.")
else:
    input_path = input_files[0]
    output_json_path = output_dir / (input_path.stem + "_parsed.json")
    output_csv_path = output_dir / (input_path.stem + "_parsed.csv")
    print(f"[INFO] Processing: {input_path.name}")
    df = process_lab_txt_file(input_path, output_json_path, output_csv_path, model)
    display(df.head())

[INFO] Found 1 txt files: ['24489525 labs.txt']
[INFO] Processing: 24489525 labs.txt
[INFO] Total lab sheets found: 25


Gemini batches:   0%|          | 0/9 [00:00<?, ?it/s]

[ERROR] Gemini API error: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 2.
[INFO] Saved raw Gemini replies: output\24489525 labs_gemini_raw_replies.txt
[INFO] Saved merged JSON: output\24489525 labs_parsed.json
[INFO] Saved flattened CSV: output\24489525 labs_parsed.csv


,patient_id,lab_order_id,encounter_id,department,specimen,lab_report_type,diagnosis_codes,requested,collected,received,reported,test_name,value,unit,reference_range,interpretation
0,24489525,T3069530,1140225-2833,感染科,SERUM,生化/血清/免疫檢驗報告,B20;B45.1;B01.0;R80.9,20250225-103419,20250225-103801,20250225-103801,20250225-141700,STS(RPR) (梅毒檢查),"1:8X, Reactive",,Non-Reactive,Reactive
1,24489525,T6631532,1140225-2833,感染科,WHOLE BLOOD,病毒檢驗報告,B20;B45.1;B01.0;R80.9,None,None,None,20250406-000000,CD3 Total T Cell,55.5,%,(49.1-83.6),normal
2,24489525,T6631532,1140225-2833,感染科,WHOLE BLOOD,病毒檢驗報告,B20;B45.1;B01.0;R80.9,None,None,None,20250406-000000,CD19 Total B Cell,9.8,%,(6.5-27.0),normal
3,24489525,T6631532,1140225-2833,感染科,WHOLE BLOOD,病毒檢驗報告,B20;B45.1;B01.0;R80.9,None,None,None,20250406-000000,CD3 CD4 Helper T cell,14.6,%,(28.2-62.8),decreased
4,24489525,T6631532,1140225-2833,感染科,WHOLE BLOOD,病毒檢驗報告,B20;B45.1;B01.0;R80.9,None,None,None,20250406-000000,CD3 CD8 Suppressor T,36.5,%,(10.2-40.1),normal


In [54]:
# Assuming previous cell extracted all reports into a variable called `total_reports`
# We'll gather all reports into a single list first.

all_reports = []
for idx, block in enumerate(blocks, 1):
    block = block.strip()
    if not block:
        continue
    reports = extract_reports(block)
    all_reports.extend(reports)

# Now analyze lab_order_id
lab_order_ids = [r.get("lab_order_id") for r in all_reports if r.get("lab_order_id")]
unique_lab_order_ids = set(lab_order_ids)

print(f"Total reports: {len(all_reports)}")
print(f"Unique lab_order_id count: {len(unique_lab_order_ids)}")
print(f"First 10 unique lab_order_id values: {list(unique_lab_order_ids)[:10]}")


Total reports: 26
Unique lab_order_id count: 21
First 10 unique lab_order_id values: ['T5005093', 'T3056563', 'T8211835', 'T5025665', 'T3047592', 'T3072294', 'T3029790', 'T3000324', 'T3053550', 'T6622738']


In [55]:
import shutil
from pathlib import Path

input_dir = Path("./input")
output_dir = Path("./output")
archive_dir = Path("./archive")
archive_dir.mkdir(exist_ok=True)

for txt_path in input_dir.glob("*.txt"):
    stem = txt_path.stem
    subfolder = archive_dir / stem
    if subfolder.exists():
        shutil.rmtree(subfolder)  # Clean up old archive if exists
    subfolder.mkdir(parents=True)
    
    # Keep track of files to delete
    files_to_delete = []
    
    # Copy the input .txt
    dest_txt = subfolder / txt_path.name
    shutil.copy(txt_path, dest_txt)
    files_to_delete.append(txt_path)
    
    # Copy all output files that match the stem
    output_files = list(output_dir.glob(f"{stem}*"))
    for out_file in output_files:
        shutil.copy(out_file, subfolder / out_file.name)
        files_to_delete.append(out_file)
    
    # Check that all files exist in the archive before deleting originals
    all_copied = all((subfolder / f.name).exists() for f in files_to_delete)
    if all_copied:
        for f in files_to_delete:
            try:
                f.unlink()
            except Exception as e:
                print(f"[WARNING] Could not delete {f}: {e}")
        print(f"[INFO] Archived and deleted originals for {stem} to {subfolder}")
    else:
        print(f"[ERROR] Not all files copied for {stem}, originals NOT deleted.")

print("[INFO] All files archived and originals deleted.")


[INFO] Archived and deleted originals for 24489525 labs to archive\24489525 labs
[INFO] All files archived and originals deleted.
